In [ ]:
'''
Python code to convert bounding boxes associated to the images of the augmented HandOverFace dataset.
The conversion is made from YOLO to format [x y width height], where x and y are the coordinates of the top left corner of the bounding box.
It is needed in order to provide the overall train dataset with the same notation used for the test dataset.
'''
# Mount the drive into Colab in order to upload datasets with bounding boxes to be converted
from google.colab import drive
drive.mount('/content/gdrive')

# Create a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive 

Mounted at /content/gdrive
 bright			    obj.names
'Colab Notebooks'	    objs
 data_augmentation.ipynb    obj.zip
 DATASET		    OneDrive_1_24-7-2022.zip
 DATASET_AUGM_DIVISO.zip   'Papiro Giulia.gdoc'
 DATASET_AUGMENTATION	   'Papiro Lucia.gdoc'
 DATASET_AUGMENTATION.zip   pred_mask
 DATASET_HANDSONFACE	    segmented_hands
 DATASET_HANDSONFACE.zip    TESTDATASET
 DATASET.zip		    TESTDATASET.zip
 DAT_HANDOVERFACE_DEF	    testobjs
 EgoHands		    test.zip
 HANDOVERFACE		    vertic_flip
 horiz_flip		    yolov4
 models			    yolov4-custom_3000.weights
'My Drive'		    yolov4-custom_4000.weights
 Notability		    yolov4-custom.cfg
 obj.data		    yolov4-custom_last.weights


In [ ]:
from natsort import natsorted
import shutil
import os
import numpy as np
import cv2

In [ ]:
# Unzip DATASET_AUGM_DIVISO, that contains both images and bounding boxes in YOLO format
%cd /mydrive
!unzip /mydrive/DATASET_AUGM_DIVISO.zip

/content/gdrive/My Drive
Archive:  /mydrive/DATASET_AUGM_DIVISO.zip
   creating: DATASET_AUGM_DIVISO/bb/
  inflating: DATASET_AUGM_DIVISO/bb/10.txt  
  inflating: DATASET_AUGM_DIVISO/bb/100.txt  
  inflating: DATASET_AUGM_DIVISO/bb/101.txt  
  inflating: DATASET_AUGM_DIVISO/bb/102.txt  
  inflating: DATASET_AUGM_DIVISO/bb/103.txt  
  inflating: DATASET_AUGM_DIVISO/bb/104.txt  
  inflating: DATASET_AUGM_DIVISO/bb/105.txt  
  inflating: DATASET_AUGM_DIVISO/bb/106.txt  
  inflating: DATASET_AUGM_DIVISO/bb/107.txt  
  inflating: DATASET_AUGM_DIVISO/bb/108.txt  
  inflating: DATASET_AUGM_DIVISO/bb/109.txt  
  inflating: DATASET_AUGM_DIVISO/bb/11.txt  
  inflating: DATASET_AUGM_DIVISO/bb/110.txt  
  inflating: DATASET_AUGM_DIVISO/bb/111.txt  
  inflating: DATASET_AUGM_DIVISO/bb/112.txt  
  inflating: DATASET_AUGM_DIVISO/bb/114.txt  
  inflating: DATASET_AUGM_DIVISO/bb/115.txt  
  inflating: DATASET_AUGM_DIVISO/bb/116.txt  
  inflating: DATASET_AUGM_DIVISO/bb/117.txt  
  inflating: DATASET_AU

In [ ]:
def dir_list(d):
    return [os.path.join(d,f) for f in os.listdir(d)] 

In [ ]:
# Get ordered paths to images and bounding boxes
testbb_path = natsorted(dir_list('/mydrive/DATASET_AUGM_DIVISO/bb'))
testimage_path = natsorted(dir_list('/mydrive/DATASET_AUGM_DIVISO/im'))

# Check if the number of elements in testbb_path and testimage_path is the same. If not, print an error
if (len(testbb_path) != len(testimage_path)):
  print('Error: not same number of images and bounding boxes')

# List all bounding boxes for each image in testimage_path. Create a list of lists.
# Each list contains the bounding boxes, relative to one of the images
testbb = []*len(testbb_path)
for testpath in testbb_path:
  testbb_img = []
  file = open(testpath, 'r')
  lines = file.readlines()
  for line in lines:
    testbb_img.append([float(x) for x in line.split()])   
  testbb.append(testbb_img) 

In [ ]:
# Obtain bounding boxes without class (first 0 element in each row of .txt files, that contain bounding boxes coordinates). 
testbb_noclass = []*len(testbb_path)
for i in range(len(testbb_path)):
    image = cv2.imread(testimage_path[i])

    bb_image = []
    for boxes in testbb[i]:
        bb_image.append(boxes[1:5])
        
    testbb_noclass.append(bb_image)

In [ ]:
def yolo2bbox(bboxes, w, h):
    """
    Function to convert bounding boxes in YOLO format to 
    xmin, ymin, width, height.
    
    Parameters:
    1 bboxes: Normalized [x_center, y_center, width, height] list
    2 w = width
    3 h = height image
    Returns: bb_list, list that contains normalized xmin, ymin, width, height
    """

    bb_list = []
    xmin, ymin = (bboxes[0]-bboxes[2]/2)*w, (bboxes[1]-bboxes[3]/2)*h
    xmax, ymax = (bboxes[0]+bboxes[2]/2)*w, (bboxes[1]+bboxes[3]/2)*h
    width = xmax - xmin
    height = ymax - ymin
    xmin = int(xmin)
    ymin = int(ymin)
    width = int(width)
    height = int(height)

    bb_list.append(xmin)
    bb_list.append(ymin)
    bb_list.append(width)
    bb_list.append(height)

    return bb_list

In [ ]:
# Create directory "HANDOVERFACE", to save images and bounding boxes in the new desired format
%cd /mydrive
!mkdir HANDOVERFACE
%cd /mydrive/HANDOVERFACE

/content/gdrive/My Drive
/content/gdrive/My Drive/HANDOVERFACE


In [ ]:
bb_img_norm = []

# Apply the function yolo2bbox to all the elements in testbb_noclass
for i in range(len(testbb_path)):

    image = cv2.imread(testimage_path[i])
    img_height, img_width, img_chann = image.shape

    # Save images and new text files into directory "HANDOVERFACE"
    filenameImg = "face_"+str(i)+".jpg";
    cv2.imwrite(filenameImg, image)

    filenameBb = "face_"+str(i)+".txt";
    f = open(filenameBb,"w+")

    bb_image = []
    for boxes in testbb_noclass[i]:
      bb_norm = yolo2bbox(boxes, img_width, img_height)  
      bb_image.append(bb_norm)
      for items in bb_norm: 
        f.write(str(items) + " ")
      f.write("\n")

    bb_img_norm.append(bb_image)
    f.close()

In [ ]:
%cd /mydrive
!mkdir DAT_HANDOVERFACE_DEF

/content/gdrive/My Drive


In [ ]:
# Zip folder "HANDOVERFACE" into directory DAT_HANDOVERFACE_DEF
%cd /mydrive/DAT_HANDOVERFACE_DEF
shutil.make_archive("HANDOVERFACE", 'zip', "/mydrive/HANDOVERFACE")

/content/gdrive/My Drive/DAT_HANDOVERFACE_DEF


'/content/gdrive/My Drive/DAT_HANDOVERFACE_DEF/HANDOVERFACE.zip'